In [3]:
import pandas as pd
import numpy as np
import re
import gensim
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from wordcloud import WordCloud
from gensim import corpora, models, similarities
import pyLDAvis
from pprint import pprint
import pickle 
import os
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel
from gensim.test.utils import datapath
import random
from gensim.models.ldamodel import LdaModel
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from gensim.corpora import MmCorpus
import csv
import pyLDAvis.gensim_models as gensimvis
from sklearn import metrics
RANDOM_STATE = 1

In [4]:
data = pd.read_csv('dataset/DatasetLegal.csv')
# train_data = pd.read_csv('model/train_data.csv')
# test_data = pd.read_csv('model/test_data.csv')
with open("model/train_data.csv", 'r') as f:
  train_data_csv = csv.reader(f)
  train_data = [i for i in train_data_csv]
with open("model/test_data.csv", 'r') as f:
  test_data_csv = csv.reader(f)
  test_data = [i for i in test_data_csv]
with open('model/id2word.pkl', 'rb') as f:
    id2word = pickle.load(f)
corpus = []
for text in train_data:
  vec = id2word.doc2bow(text)
  corpus.append(vec)
pd.Series(corpus)
with open("model/topic_dict.pkl", "rb") as f:
   topic_dict = pickle.load(f)
with open("model/lda_model.pkl", "rb") as f:
   lda_model = pickle.load(f)

import logging
logging.basicConfig(filename='model_callbacks.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.NOTSET)

from gensim.models.callbacks import PerplexityMetric, CoherenceMetric
perplexity_logger = PerplexityMetric(corpus=corpus, logger='shell')
coherence_cv_logger = CoherenceMetric(corpus=corpus, logger='shell', coherence = 'c_v', texts = train_data)

with open("model/corpus_question.pkl", "rb") as f:
   corpus_question = pickle.load(f)
with open("model/corpus_answer.pkl", "rb") as f:
   corpus_answer = pickle.load(f)

corpus_question_train = corpus_question[:5625]
corpus_question_test = corpus_question[5625:]
corpus_answer_train = corpus_answer[:5625]
corpus_answer_test = corpus_answer[5625:]

num_topics = 4

In [5]:
with open("lda/lda_model_4.pkl", "rb") as f:
    lda_model_4 = pickle.load(f)
with open(f"lda/topic_dict_4.pkl", "rb") as f:
    topic_dict_4 = pickle.load(f)
num_topics = len(topic_dict_4)
pred_question_score = [lda_model_4[text] for text in corpus_question_train]
pred_answer_score = [lda_model_4[text] for text in corpus_answer_train]
threshold = 1/num_topics
question_predict=[]
for each_topic in pred_question_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold): temp_pred.append(1)
    else: temp_pred.append(0)
  question_predict.append(temp_pred)
pd.DataFrame(question_predict)
answer_predict=[]
for each_topic in pred_answer_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold) : temp_pred.append(1)
    else: temp_pred.append(0)
  answer_predict.append(temp_pred)
pd.DataFrame(answer_predict)
y_true = np.array(answer_predict)
y_pred = np.array(question_predict)
topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]

print(f'accuracy for topic {topic_dict_4[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {topic_dict_4[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {topic_dict_4[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {topic_dict_4[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')

accuracy for topic labor : 0.74
accuracy for topic personal rights/court : 0.92
accuracy for topic family/succession : 0.94
accuracy for topic contract : 0.62


In [11]:
with open("lda/lda_model_4.pkl", "rb") as f:
    lda_model_4 = pickle.load(f)
with open(f"lda/topic_dict_4.pkl", "rb") as f:
    topic_dict_4 = pickle.load(f)
num_topics = len(topic_dict_4)
pred_question_score = [lda_model_4[text] for text in corpus_question_test]
pred_answer_score = [lda_model_4[text] for text in corpus_answer_test]
threshold = 1/num_topics
question_predict=[]
for each_topic in pred_question_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold): temp_pred.append(1)
    else: temp_pred.append(0)
  question_predict.append(temp_pred)
pd.DataFrame(question_predict)
answer_predict=[]
for each_topic in pred_answer_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold) : temp_pred.append(1)
    else: temp_pred.append(0)
  answer_predict.append(temp_pred)
pd.DataFrame(answer_predict)
y_true = np.array(answer_predict)
y_pred = np.array(question_predict)
topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]

print(f'accuracy for topic {topic_dict_4[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {topic_dict_4[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {topic_dict_4[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {topic_dict_4[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')

accuracy for topic labor : 0.72
accuracy for topic personal rights/court : 0.93
accuracy for topic family/succession : 0.93
accuracy for topic contract : 0.57


In [6]:
y_true = answer_predict
y_pred = question_predict

count_true = 0
count_false = 0
pred_list = []
true_list = []
checker = False
for i in range(len(y_true)):
    checker = False
    for j in range(num_topics):
        if(y_true[i][j] == 1):
            if(y_true[i][j] == y_pred[i][j]):
                checker = True
    if(checker == True):
        count_true += 1
        pred_list.append(y_pred[i])
        true_list.append(y_true[i])
    if(checker == False):
        count_false += 1
print(f'count_true = {count_true} docs\ncount_false = {count_false} docs')
print(f"accuracy = {count_true/(count_true+count_false):.2f}")

count_true = 4701 docs
count_false = 924 docs
accuracy = 0.84


In [12]:
y_true = answer_predict
y_pred = question_predict

count_true = 0
count_false = 0
pred_list = []
true_list = []
checker = False
for i in range(len(y_true)):
    checker = False
    for j in range(num_topics):
        if(y_true[i][j] == 1):
            if(y_true[i][j] == y_pred[i][j]):
                checker = True
    if(checker == True):
        count_true += 1
        pred_list.append(y_pred[i])
        true_list.append(y_true[i])
    if(checker == False):
        count_false += 1
print('test')
print(f'count_true = {count_true} docs\ncount_false = {count_false} docs')
print(f"accuracy = {count_true/(count_true+count_false):.2f}")

test
count_true = 1160 docs
count_false = 246 docs
accuracy = 0.83


In [7]:
y_true = answer_predict
y_pred = question_predict

count_true = 0
count_false = 0
pred_list = []
true_list = []
checker = False
for i in range(len(y_true)):
    checker = False
    for j in range(num_topics):
        if(y_true[i][j] == 1):
            if(y_true[i][j] == y_pred[i][j]):
                checker = True
    if(checker == True):
        count_true += 1
        pred_list.append(y_pred[i])
        true_list.append(y_true[i])
    if(checker == False):
        count_false += 1
print(f'count_true = {count_true} docs\ncount_false = {count_false} docs')
print(f"accuracy = {count_true/(count_true+count_false):.2f}")

count_true = 4701 docs
count_false = 924 docs
accuracy = 0.84


In [8]:
y_true = answer_predict
y_pred = question_predict
accuracy_list_bleu = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_pred[i][j]==1):
            count += 1
            if(y_true[i][j]==1):
                check += 1
    accuracy_list_bleu.append(check/count)
avg_acc_list_bleu = sum(accuracy_list_bleu)/len(y_pred)
print(f"accuracy (bleu) = {avg_acc_list_bleu:.2f}")

accuracy_list_rouge = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_true[i][j]==1):
            count += 1
            if(y_pred[i][j]==1):
                check += 1
    accuracy_list_rouge.append(check/count)
avg_acc_list_rouge = sum(accuracy_list_rouge)/len(y_pred)
print(f"accuracy (rouge) = {avg_acc_list_rouge:.2f}")

accuraccy_list_jacc = []
for i in range(len(y_pred)):
    jacc = 0
    for j in range(num_topics):
        if(y_true[i][j]==y_pred[i][j]): jacc+=1
    accuraccy_list_jacc.append(jacc/num_topics)
avg_acc_list_jacc = sum(accuraccy_list_jacc)/len(y_pred)
print(f"accuracy (jaccard) = {avg_acc_list_jacc:.2f}")

accuracy (bleu) = 0.70
accuracy (rouge) = 0.77
accuracy (jaccard) = 0.81


In [13]:
y_true = answer_predict
y_pred = question_predict
print('test')
accuracy_list_bleu = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_pred[i][j]==1):
            count += 1
            if(y_true[i][j]==1):
                check += 1
    accuracy_list_bleu.append(check/count)
avg_acc_list_bleu = sum(accuracy_list_bleu)/len(y_pred)
print(f"accuracy (bleu) = {avg_acc_list_bleu:.2f}")

accuracy_list_rouge = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_true[i][j]==1):
            count += 1
            if(y_pred[i][j]==1):
                check += 1
    accuracy_list_rouge.append(check/count)
avg_acc_list_rouge = sum(accuracy_list_rouge)/len(y_pred)
print(f"accuracy (rouge) = {avg_acc_list_rouge:.2f}")

accuraccy_list_jacc = []
for i in range(len(y_pred)):
    jacc = 0
    for j in range(num_topics):
        if(y_true[i][j]==y_pred[i][j]): jacc+=1
    accuraccy_list_jacc.append(jacc/num_topics)
avg_acc_list_jacc = sum(accuraccy_list_jacc)/len(y_pred)
print(f"accuracy (jaccard) = {avg_acc_list_jacc:.2f}")

test
accuracy (bleu) = 0.68
accuracy (rouge) = 0.75
accuracy (jaccard) = 0.79


In [9]:
metrics.multilabel_confusion_matrix(y_true, y_pred)

array([[[4194,  223],
        [ 229,  979]],

       [[2598,  212],
        [1240, 1575]],

       [[3533,  136],
        [ 207, 1749]],

       [[2803, 2006],
        [ 131,  685]]], dtype=int64)

In [14]:
print('test')
metrics.multilabel_confusion_matrix(y_true, y_pred)

test


array([[[1090,   51],
        [  48,  217]],

       [[ 597,   42],
        [ 357,  410]],

       [[ 872,   32],
        [  60,  442]],

       [[ 644,  570],
        [  28,  164]]], dtype=int64)

In [10]:
label_names = list(topic_dict_4.values())
print(metrics.classification_report(y_true, y_pred,target_names=label_names))

                       precision    recall  f1-score   support

                labor       0.81      0.81      0.81      1208
personal rights/court       0.88      0.56      0.68      2815
    family/succession       0.93      0.89      0.91      1956
             contract       0.25      0.84      0.39       816

            micro avg       0.66      0.73      0.69      6795
            macro avg       0.72      0.78      0.70      6795
         weighted avg       0.81      0.73      0.74      6795
          samples avg       0.70      0.77      0.71      6795



In [15]:
label_names = list(topic_dict_4.values())
print('test')
print(metrics.classification_report(y_true, y_pred,target_names=label_names))

test
                       precision    recall  f1-score   support

                labor       0.81      0.82      0.81       265
personal rights/court       0.91      0.53      0.67       767
    family/succession       0.93      0.88      0.91       502
             contract       0.22      0.85      0.35       192

            micro avg       0.64      0.71      0.67      1726
            macro avg       0.72      0.77      0.69      1726
         weighted avg       0.82      0.71      0.73      1726
          samples avg       0.68      0.75      0.68      1726



num_topics = 5

In [16]:
with open("lda/lda_model_5.pkl", "rb") as f:
    lda_model_5 = pickle.load(f)
with open(f"lda/topic_dict_5.pkl", "rb") as f:
    topic_dict_5 = pickle.load(f)
num_topics = len(topic_dict_5)
pred_question_score = [lda_model_5[text] for text in corpus_question_train]
pred_answer_score = [lda_model_5[text] for text in corpus_answer_train]
threshold = 1/num_topics
question_predict=[]
for each_topic in pred_question_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold): temp_pred.append(1)
    else: temp_pred.append(0)
  question_predict.append(temp_pred)
pd.DataFrame(question_predict)
answer_predict=[]
for each_topic in pred_answer_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold) : temp_pred.append(1)
    else: temp_pred.append(0)
  answer_predict.append(temp_pred)
pd.DataFrame(answer_predict)
y_true = np.array(answer_predict)
y_pred = np.array(question_predict)
topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]
topic_true_4 = [sublist[4] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]
topic_pred_4 = [sublist[4] for sublist in y_pred]

print(f'accuracy for topic {topic_dict_5[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {topic_dict_5[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {topic_dict_5[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {topic_dict_5[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')
print(f'accuracy for topic {topic_dict_5[4]} : {metrics.accuracy_score(topic_true_4, topic_pred_4):.2f}')

accuracy for topic labor : 0.69
accuracy for topic personal_right/court : 0.91
accuracy for topic family/succession : 0.93
accuracy for topic contract : 0.73
accuracy for topic criminal : 0.75


In [28]:
df = pd.DataFrame(question_predict)
df.columns = ["labor", "perso...", "family...", "contract", "criminal"]
df

,labor,perso...,family...,contract,criminal
0,0,0,1,0,0
1,0,0,1,0,0
2,0,1,0,0,0
3,0,0,1,0,0
4,1,0,0,1,0
...,...,...,...,...,...
5620,1,1,0,1,0
5621,0,1,0,1,0
5622,0,0,0,1,1
5623,0,0,1,0,1


In [29]:
df= pd.DataFrame(answer_predict)
df.columns = ["labor", "perso...", "family...", "contract", "criminal"]
df

,labor,perso...,family...,contract,criminal
0,0,0,1,0,0
1,0,0,1,0,0
2,0,1,0,0,0
3,0,0,1,0,0
4,0,0,0,1,0
...,...,...,...,...,...
5620,0,1,0,0,0
5621,0,1,0,0,1
5622,0,1,0,0,1
5623,0,1,1,0,0


In [6]:
with open("lda/lda_model_5.pkl", "rb") as f:
    lda_model_5 = pickle.load(f)
with open(f"lda/topic_dict_5.pkl", "rb") as f:
    topic_dict_5 = pickle.load(f)
num_topics = len(topic_dict_5)
pred_question_score = [lda_model_5[text] for text in corpus_question_test]
pred_answer_score = [lda_model_5[text] for text in corpus_answer_test]
threshold = 1/num_topics
question_predict=[]
for each_topic in pred_question_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold): temp_pred.append(1)
    else: temp_pred.append(0)
  question_predict.append(temp_pred)
pd.DataFrame(question_predict)
answer_predict=[]
for each_topic in pred_answer_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold) : temp_pred.append(1)
    else: temp_pred.append(0)
  answer_predict.append(temp_pred)
pd.DataFrame(answer_predict)
y_true = np.array(answer_predict)
y_pred = np.array(question_predict)
topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]
topic_true_4 = [sublist[4] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]
topic_pred_4 = [sublist[4] for sublist in y_pred]
print('test')
print(f'accuracy for topic {topic_dict_5[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {topic_dict_5[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {topic_dict_5[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {topic_dict_5[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')
print(f'accuracy for topic {topic_dict_5[4]} : {metrics.accuracy_score(topic_true_4, topic_pred_4):.2f}')

test
accuracy for topic labor : 0.67
accuracy for topic personal_right/court : 0.92
accuracy for topic family/succession : 0.93
accuracy for topic contract : 0.72
accuracy for topic criminal : 0.75


In [42]:
y_true = answer_predict
y_pred = question_predict

count_true = 0
count_false = 0
pred_list = []
true_list = []
checker = False
for i in range(len(y_true)):
    checker = False
    for j in range(num_topics):
        if(y_true[i][j] == 1):
            if(y_true[i][j] == y_pred[i][j]):
                checker = True
    if(checker == True):
        count_true += 1
        pred_list.append(y_pred[i])
        true_list.append(y_true[i])
    if(checker == False):
        count_false += 1
print(f'count_true = {count_true} docs\ncount_false = {count_false} docs')
print(f"accuracy = {count_true/(count_true+count_false):.2f}")

count_true = 4691 docs
count_false = 934 docs
accuracy = 0.83


In [5]:
y_true = answer_predict
y_pred = question_predict

count_true = 0
count_false = 0
pred_list = []
true_list = []
checker = False
for i in range(len(y_true)):
    checker = False
    for j in range(num_topics):
        if(y_true[i][j] == 1):
            if(y_true[i][j] == y_pred[i][j]):
                checker = True
    if(checker == True):
        count_true += 1
        pred_list.append(y_pred[i])
        true_list.append(y_true[i])
    if(checker == False):
        count_false += 1
print('test')
print(f'count_true = {count_true} docs\ncount_false = {count_false} docs')
print(f"accuracy = {count_true/(count_true+count_false):.2f}")

test
count_true = 1205 docs
count_false = 201 docs
accuracy = 0.86


In [40]:
y_true = answer_predict
y_pred = question_predict

count_true = 0
count_false = 0
pred_list = []
true_list = []
checker = False
for i in range(len(y_true)):
    checker = False
    for j in range(num_topics):
        if(y_true[i][j] == 1):
            if(y_true[i][j] == y_pred[i][j]):
                checker = True
    if(checker == True):
        count_true += 1
        pred_list.append(y_pred[i])
        true_list.append(y_true[i])
    if(checker == False):
        count_false += 1
print(f'count_true = {count_true} docs\ncount_false = {count_false} docs')
print(f"accuracy = {count_true/(count_true+count_false):.2f}")

count_true = 4691 docs
count_false = 934 docs
accuracy = 0.83


In [38]:
y_true = answer_predict
y_pred = question_predict
accuracy_list_bleu = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_pred[i][j]==1):
            count += 1
            if(y_true[i][j]==1):
                check += 1
    accuracy_list_bleu.append(check/count)
avg_acc_list_bleu = sum(accuracy_list_bleu)/len(y_pred)
print(f"accuracy (bleu) = {avg_acc_list_bleu:.2f}")

accuracy_list_rouge = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_true[i][j]==1):
            count += 1
            if(y_pred[i][j]==1):
                check += 1
    accuracy_list_rouge.append(check/count)
avg_acc_list_rouge = sum(accuracy_list_rouge)/len(y_pred)
print(f"accuracy (rouge) = {avg_acc_list_rouge:.2f}")

accuraccy_list_jacc = []
for i in range(len(y_pred)):
    jacc = 0
    for j in range(num_topics):
        if(y_true[i][j]==y_pred[i][j]): jacc+=1
    accuraccy_list_jacc.append(jacc/num_topics)
avg_acc_list_jacc = sum(accuraccy_list_jacc)/len(y_pred)
print(f"accuracy (jaccard) = {avg_acc_list_jacc:.2f}")

accuracy (bleu) = 0.66
accuracy (rouge) = 0.73
accuracy (jaccard) = 0.80


In [7]:
y_true = answer_predict
y_pred = question_predict
accuracy_list_bleu = []
print('test')
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_pred[i][j]==1):
            count += 1
            if(y_true[i][j]==1):
                check += 1
    accuracy_list_bleu.append(check/count)
avg_acc_list_bleu = sum(accuracy_list_bleu)/len(y_pred)
print(f"accuracy (bleu) = {avg_acc_list_bleu:.2f}")

accuracy_list_rouge = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_true[i][j]==1):
            count += 1
            if(y_pred[i][j]==1):
                check += 1
    accuracy_list_rouge.append(check/count)
avg_acc_list_rouge = sum(accuracy_list_rouge)/len(y_pred)
print(f"accuracy (rouge) = {avg_acc_list_rouge:.2f}")

accuraccy_list_jacc = []
for i in range(len(y_pred)):
    jacc = 0
    for j in range(num_topics):
        if(y_true[i][j]==y_pred[i][j]): jacc+=1
    accuraccy_list_jacc.append(jacc/num_topics)
avg_acc_list_jacc = sum(accuraccy_list_jacc)/len(y_pred)
print(f"accuracy (jaccard) = {avg_acc_list_jacc:.2f}")

test
accuracy (bleu) = 0.68
accuracy (rouge) = 0.73
accuracy (jaccard) = 0.80


In [10]:
metrics.multilabel_confusion_matrix(y_true, y_pred)

array([[[4159,  229],
        [ 283,  954]],

       [[2724,  205],
        [1541, 1155]],

       [[3488,  162],
        [ 224, 1751]],

       [[3283, 1360],
        [ 177,  805]],

       [[3746, 1288],
        [ 146,  445]]], dtype=int64)

In [8]:
print('test')
metrics.multilabel_confusion_matrix(y_true, y_pred)

test


array([[[1080,   47],
        [  59,  220]],

       [[ 636,   45],
        [ 423,  302]],

       [[ 850,   41],
        [  60,  455]],

       [[ 806,  350],
        [  38,  212]],

       [[ 901,  334],
        [  15,  156]]], dtype=int64)

In [11]:
label_names = list(topic_dict_5.values())
print(metrics.classification_report(y_true, y_pred,target_names=label_names))

                      precision    recall  f1-score   support

               labor       0.81      0.77      0.79      1237
personal_right/court       0.85      0.43      0.57      2696
   family/succession       0.92      0.89      0.90      1975
            contract       0.37      0.82      0.51       982
            criminal       0.26      0.75      0.38       591

           micro avg       0.61      0.68      0.65      7481
           macro avg       0.64      0.73      0.63      7481
        weighted avg       0.75      0.68      0.67      7481
         samples avg       0.66      0.73      0.66      7481



In [9]:
label_names = list(topic_dict_5.values())
print('test')
print(metrics.classification_report(y_true, y_pred,target_names=label_names))

test
                      precision    recall  f1-score   support

               labor       0.82      0.79      0.81       279
personal_right/court       0.87      0.42      0.56       725
   family/succession       0.92      0.88      0.90       515
            contract       0.38      0.85      0.52       250
            criminal       0.32      0.91      0.47       171

           micro avg       0.62      0.69      0.66      1940
           macro avg       0.66      0.77      0.65      1940
        weighted avg       0.76      0.69      0.67      1940
         samples avg       0.68      0.73      0.67      1940



num_topics = 6

In [43]:
with open("lda/lda_model_6.pkl", "rb") as f:
    lda_model_6 = pickle.load(f)
with open(f"lda/topic_dict_6.pkl", "rb") as f:
    topic_dict_6 = pickle.load(f)
num_topics = len(topic_dict_6)
pred_question_score = [lda_model_6[text] for text in corpus_question_train]
pred_answer_score = [lda_model_6[text] for text in corpus_answer_train]
threshold = 1/num_topics
question_predict=[]
for each_topic in pred_question_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold): temp_pred.append(1)
    else: temp_pred.append(0)
  question_predict.append(temp_pred)
pd.DataFrame(question_predict)
answer_predict=[]
for each_topic in pred_answer_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold) : temp_pred.append(1)
    else: temp_pred.append(0)
  answer_predict.append(temp_pred)
pd.DataFrame(answer_predict)
y_true = np.array(answer_predict)
y_pred = np.array(question_predict)
topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]
topic_true_4 = [sublist[4] for sublist in y_true]
topic_true_5 = [sublist[5] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]
topic_pred_4 = [sublist[4] for sublist in y_pred]
topic_pred_5 = [sublist[5] for sublist in y_pred]

print(f'accuracy for topic {topic_dict_6[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {topic_dict_6[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {topic_dict_6[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {topic_dict_6[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')
print(f'accuracy for topic {topic_dict_6[4]} : {metrics.accuracy_score(topic_true_4, topic_pred_4):.2f}')
print(f'accuracy for topic {topic_dict_6[5]} : {metrics.accuracy_score(topic_true_5, topic_pred_5):.2f}')

accuracy for topic labor : 0.70
accuracy for topic personal_right/court : 0.72
accuracy for topic family/succession : 0.93
accuracy for topic contract : 0.79
accuracy for topic criminal : 0.74
accuracy for topic labor : 0.72


In [44]:
y_true = answer_predict
y_pred = question_predict

count_true = 0
count_false = 0
pred_list = []
true_list = []
checker = False
for i in range(len(y_true)):
    checker = False
    for j in range(num_topics):
        if(y_true[i][j] == 1):
            if(y_true[i][j] == y_pred[i][j]):
                checker = True
    if(checker == True):
        count_true += 1
        pred_list.append(y_pred[i])
        true_list.append(y_true[i])
    if(checker == False):
        count_false += 1
print(f'count_true = {count_true} docs\ncount_false = {count_false} docs')
print(f"accuracy = {count_true/(count_true+count_false):.2f}")

count_true = 4357 docs
count_false = 1268 docs
accuracy = 0.77


In [32]:
y_true = answer_predict
y_pred = question_predict
accuracy_list_bleu = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_pred[i][j]==1):
            count += 1
            if(y_true[i][j]==1):
                check += 1
    accuracy_list_bleu.append(check/count)
avg_acc_list_bleu = sum(accuracy_list_bleu)/len(y_pred)
print(f"accuracy (bleu) = {avg_acc_list_bleu:.2f}")
accuracy_list_rouge = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_true[i][j]==1):
            count += 1
            if(y_pred[i][j]==1):
                check += 1
    accuracy_list_rouge.append(check/count)
avg_acc_list_rouge = sum(accuracy_list_rouge)/len(y_pred)
print(f"accuracy (rouge) = {avg_acc_list_rouge:.2f}")

accuraccy_list_jacc = []
for i in range(len(y_pred)):
    jacc = 0
    for j in range(num_topics):
        if(y_true[i][j]==y_pred[i][j]): jacc+=1
    accuraccy_list_jacc.append(jacc/num_topics)
avg_acc_list_jacc = sum(accuraccy_list_jacc)/len(y_pred)
print(f"accuracy (jaccard) = {avg_acc_list_jacc:.2f}")

accuracy (bleu) = 0.58
accuracy (rouge) = 0.61
accuracy (jaccard) = 0.77


In [57]:
metrics.multilabel_confusion_matrix(y_true, y_pred)

array([[[3387, 1395],
        [ 178,  665]],

       [[3173,  244],
        [1452,  756]],

       [[3450,  188],
        [ 221, 1766]],

       [[3369,  970],
        [ 237, 1049]],

       [[3704, 1335],
        [ 127,  459]],

       [[3854,   53],
        [1494,  224]]], dtype=int64)

In [58]:
label_names = list(topic_dict_6.values())
print(metrics.classification_report(y_true, y_pred,target_names=label_names))

                      precision    recall  f1-score   support

               labor       0.32      0.79      0.46       843
personal_right/court       0.76      0.34      0.47      2208
   family/succession       0.90      0.89      0.90      1987
            contract       0.52      0.82      0.63      1286
            criminal       0.26      0.78      0.39       586
               labor       0.81      0.13      0.22      1718

           micro avg       0.54      0.57      0.55      8628
           macro avg       0.59      0.62      0.51      8628
        weighted avg       0.69      0.57      0.54      8628
         samples avg       0.58      0.61      0.56      8628



In [59]:
for i in range(len(y_pred)):
    if(y_pred[i][1]==0): y_pred[i][0]=y_pred[i][5]
if(len(y_pred[i]!=5)):
    y_pred = np.delete(y_pred, 5, axis=1)
y_pred

array([[0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 0, 1]])

In [60]:
for i in range(len(y_true)):
    if(y_true[i][1]==0): y_true[i][1]=y_true[i][5]
if(len(y_true[i]!=5)):
    y_true = np.delete(y_true, 5, axis=1)
y_true

array([[0, 1, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 1],
       [0, 1, 1, 0, 1],
       [0, 1, 0, 0, 0]])

In [61]:
topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]
topic_true_4 = [sublist[4] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]
topic_pred_4 = [sublist[4] for sublist in y_pred]

print(f'accuracy for topic {topic_dict_6[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {topic_dict_6[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {topic_dict_6[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {topic_dict_6[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')
print(f'accuracy for topic {topic_dict_6[4]} : {metrics.accuracy_score(topic_true_4, topic_pred_4):.2f}')

accuracy for topic labor : 0.50
accuracy for topic personal_right/court : 0.81
accuracy for topic family/succession : 0.93
accuracy for topic contract : 0.79
accuracy for topic criminal : 0.74


In [62]:
metrics.multilabel_confusion_matrix(y_true, y_pred)
label_names = list(topic_dict_6.values())
label_names = label_names[:5]
print(metrics.classification_report(y_true, y_pred,target_names=label_names))

                      precision    recall  f1-score   support

               labor       0.27      0.17      0.21       843
personal_right/court       0.87      0.25      0.38      3542
   family/succession       0.90      0.89      0.90      1987
            contract       0.52      0.82      0.63      1286
            criminal       0.26      0.78      0.39       586

           micro avg       0.59      0.52      0.55      8244
           macro avg       0.56      0.58      0.50      8244
        weighted avg       0.72      0.52      0.53      8244
         samples avg       0.55      0.55      0.52      8244



c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


num_topics = 7

In [45]:
with open("lda/lda_model_7.pkl", "rb") as f:
    lda_model_7 = pickle.load(f)
with open(f"lda/topic_dict_7.pkl", "rb") as f:
    topic_dict_7 = pickle.load(f)
num_topics = len(topic_dict_7)
pred_question_score = [lda_model_7[text] for text in corpus_question_train]
pred_answer_score = [lda_model_7[text] for text in corpus_answer_train]
threshold = 1/num_topics
question_predict=[]
for each_topic in pred_question_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold): temp_pred.append(1)
    else: temp_pred.append(0)
  question_predict.append(temp_pred)
pd.DataFrame(question_predict)
answer_predict=[]
for each_topic in pred_answer_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold) : temp_pred.append(1)
    else: temp_pred.append(0)
  answer_predict.append(temp_pred)
pd.DataFrame(answer_predict)
y_true = np.array(answer_predict)
y_pred = np.array(question_predict)
topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]
topic_true_4 = [sublist[4] for sublist in y_true]
topic_true_5 = [sublist[5] for sublist in y_true]
topic_true_6 = [sublist[6] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]
topic_pred_4 = [sublist[4] for sublist in y_pred]
topic_pred_5 = [sublist[5] for sublist in y_pred]
topic_pred_6 = [sublist[6] for sublist in y_pred]

print(f'accuracy for topic {topic_dict_7[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {topic_dict_7[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {topic_dict_7[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {topic_dict_7[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')
print(f'accuracy for topic {topic_dict_7[4]} : {metrics.accuracy_score(topic_true_4, topic_pred_4):.2f}')
print(f'accuracy for topic {topic_dict_7[5]} : {metrics.accuracy_score(topic_true_5, topic_pred_5):.2f}')
print(f'accuracy for topic {topic_dict_7[6]} : {metrics.accuracy_score(topic_true_6, topic_pred_6):.2f}')

accuracy for topic labor : 0.73
accuracy for topic personal_right/court : 0.64
accuracy for topic family/succession : 0.93
accuracy for topic contract : 0.83
accuracy for topic criminal : 0.68
accuracy for topic contract : 0.74
accuracy for topic labor : 0.87


In [46]:
y_true = answer_predict
y_pred = question_predict

count_true = 0
count_false = 0
pred_list = []
true_list = []
checker = False
for i in range(len(y_true)):
    checker = False
    for j in range(num_topics):
        if(y_true[i][j] == 1):
            if(y_true[i][j] == y_pred[i][j]):
                checker = True
    if(checker == True):
        count_true += 1
        pred_list.append(y_pred[i])
        true_list.append(y_true[i])
    if(checker == False):
        count_false += 1
print(f'count_true = {count_true} docs\ncount_false = {count_false} docs')
print(f"accuracy = {count_true/(count_true+count_false):.2f}")

count_true = 4410 docs
count_false = 1215 docs
accuracy = 0.78


In [36]:
y_true = answer_predict
y_pred = question_predict
accuracy_list_bleu = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_pred[i][j]==1):
            count += 1
            if(y_true[i][j]==1):
                check += 1
    accuracy_list_bleu.append(check/count)
avg_acc_list_bleu = sum(accuracy_list_bleu)/len(y_pred)
print(f"accuracy (bleu) = {avg_acc_list_bleu:.2f}")
accuracy_list_rouge = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_true[i][j]==1):
            count += 1
            if(y_pred[i][j]==1):
                check += 1
    accuracy_list_rouge.append(check/count)
avg_acc_list_rouge = sum(accuracy_list_rouge)/len(y_pred)
print(f"accuracy (rouge) = {avg_acc_list_rouge:.2f}")

accuraccy_list_jacc = []
for i in range(len(y_pred)):
    jacc = 0
    for j in range(num_topics):
        if(y_true[i][j]==y_pred[i][j]): jacc+=1
    accuraccy_list_jacc.append(jacc/num_topics)
avg_acc_list_jacc = sum(accuraccy_list_jacc)/len(y_pred)
print(f"accuracy (jaccard) = {avg_acc_list_jacc:.2f}")

accuracy (bleu) = 0.55
accuracy (rouge) = 0.60
accuracy (jaccard) = 0.77


In [70]:
metrics.multilabel_confusion_matrix(y_true, y_pred)
label_names = list(topic_dict_7.values())
print(metrics.classification_report(y_true, y_pred,target_names=label_names))

                      precision    recall  f1-score   support

               labor       0.25      0.80      0.38       795
personal_right/court       0.79      0.34      0.48      1995
   family/succession       0.88      0.92      0.90      1942
            contract       0.58      0.74      0.65      1213
            criminal       0.21      0.74      0.33       581
            contract       0.74      0.13      0.22      1626
               labor       0.77      0.40      0.53      1036

           micro avg       0.52      0.55      0.53      9188
           macro avg       0.60      0.58      0.50      9188
        weighted avg       0.69      0.55      0.53      9188
         samples avg       0.55      0.60      0.54      9188



In [71]:
for i in range(len(y_pred)):
    if(y_pred[i][0]==0): y_pred[i][1]=y_pred[i][6]
    if(y_pred[i][3]==0): y_pred[i][1]=y_pred[i][5]
if(len(y_pred[i]!=5)):
    y_pred = np.delete(y_pred, 6, axis=1)
    y_pred = np.delete(y_pred, 5, axis=1)
for i in range(len(y_true)):
    if(y_true[i][0]==0): y_true[i][1]=y_true[i][6]
    if(y_true[i][3]==0): y_true[i][1]=y_true[i][5]
if(len(y_true[i]!=5)):
    y_true = np.delete(y_true, 6, axis=1)
    y_true = np.delete(y_true, 5, axis=1)

In [72]:
topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]
topic_true_4 = [sublist[4] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]
topic_pred_4 = [sublist[4] for sublist in y_pred]

print(f'accuracy for topic {topic_dict_6[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {topic_dict_6[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {topic_dict_6[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {topic_dict_6[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')
print(f'accuracy for topic {topic_dict_6[4]} : {metrics.accuracy_score(topic_true_4, topic_pred_4):.2f}')

accuracy for topic labor : 0.77
accuracy for topic personal_right/court : 0.64
accuracy for topic family/succession : 0.93
accuracy for topic contract : 0.83
accuracy for topic criminal : 0.68


In [73]:
metrics.multilabel_confusion_matrix(y_true, y_pred)
label_names = list(topic_dict_7.values())
label_names = label_names[:5]
print(metrics.classification_report(y_true, y_pred,target_names=label_names))

                      precision    recall  f1-score   support

               labor       0.25      0.80      0.38       795
personal_right/court       0.53      0.13      0.22      1290
   family/succession       0.88      0.92      0.90      1942
            contract       0.58      0.74      0.65      1213
            criminal       0.21      0.74      0.33       581

           micro avg       0.46      0.67      0.55      5821
           macro avg       0.49      0.67      0.50      5821
        weighted avg       0.59      0.67      0.57      5821
         samples avg       0.48      0.55      0.49      5821



c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


num_topics = 8

In [47]:
with open("lda/lda_model_8.pkl", "rb") as f:
    lda_model_8 = pickle.load(f)
with open(f"lda/topic_dict_8.pkl", "rb") as f:
    topic_dict_8 = pickle.load(f)
num_topics = len(topic_dict_8)
pred_question_score = [lda_model_8[text] for text in corpus_question_train]
pred_answer_score = [lda_model_8[text] for text in corpus_answer_train]
threshold = 1/num_topics
question_predict=[]
for each_topic in pred_question_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold): temp_pred.append(1)
    else: temp_pred.append(0)
  question_predict.append(temp_pred)
pd.DataFrame(question_predict)
answer_predict=[]
for each_topic in pred_answer_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold) : temp_pred.append(1)
    else: temp_pred.append(0)
  answer_predict.append(temp_pred)
pd.DataFrame(answer_predict)
y_true = np.array(answer_predict)
y_pred = np.array(question_predict)
topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]
topic_true_4 = [sublist[4] for sublist in y_true]
topic_true_5 = [sublist[5] for sublist in y_true]
topic_true_6 = [sublist[6] for sublist in y_true]
topic_true_7 = [sublist[7] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]
topic_pred_4 = [sublist[4] for sublist in y_pred]
topic_pred_5 = [sublist[5] for sublist in y_pred]
topic_pred_6 = [sublist[6] for sublist in y_pred]
topic_pred_7 = [sublist[7] for sublist in y_pred]

print(f'accuracy for topic {topic_dict_8[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {topic_dict_8[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {topic_dict_8[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {topic_dict_8[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')
print(f'accuracy for topic {topic_dict_8[4]} : {metrics.accuracy_score(topic_true_4, topic_pred_4):.2f}')
print(f'accuracy for topic {topic_dict_8[5]} : {metrics.accuracy_score(topic_true_5, topic_pred_5):.2f}')
print(f'accuracy for topic {topic_dict_8[6]} : {metrics.accuracy_score(topic_true_6, topic_pred_6):.2f}')
print(f'accuracy for topic {topic_dict_8[7]} : {metrics.accuracy_score(topic_true_7, topic_pred_7):.2f}')

accuracy for topic contract : 0.72
accuracy for topic personal_right/court : 0.67
accuracy for topic family/succession : 0.93
accuracy for topic contract : 0.76
accuracy for topic criminal : 0.78
accuracy for topic contract : 0.73
accuracy for topic labor : 0.88
accuracy for topic labor : 0.83


In [48]:
y_true = answer_predict
y_pred = question_predict

count_true = 0
count_false = 0
pred_list = []
true_list = []
checker = False
for i in range(len(y_true)):
    checker = False
    for j in range(num_topics):
        if(y_true[i][j] == 1):
            if(y_true[i][j] == y_pred[i][j]):
                checker = True
    if(checker == True):
        count_true += 1
        pred_list.append(y_pred[i])
        true_list.append(y_true[i])
    if(checker == False):
        count_false += 1
print(f'count_true = {count_true} docs\ncount_false = {count_false} docs')
print(f"accuracy = {count_true/(count_true+count_false):.2f}")

count_true = 4548 docs
count_false = 1077 docs
accuracy = 0.81


In [24]:
y_true = answer_predict
y_pred = question_predict
accuracy_list_bleu = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_pred[i][j]==1):
            count += 1
            if(y_true[i][j]==1):
                check += 1
    accuracy_list_bleu.append(check/count)
avg_acc_list_bleu = sum(accuracy_list_bleu)/len(y_pred)
print(f"accuracy (bleu) = {avg_acc_list_bleu:.2f}")
accuracy_list_rouge = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_true[i][j]==1):
            count += 1
            if(y_pred[i][j]==1):
                check += 1
    accuracy_list_rouge.append(check/count)
avg_acc_list_rouge = sum(accuracy_list_rouge)/len(y_pred)
print(f"accuracy (rouge) = {avg_acc_list_rouge:.2f}")

accuraccy_list_jacc = []
for i in range(len(y_pred)):
    jacc = 0
    for j in range(num_topics):
        if(y_true[i][j]==y_pred[i][j]): jacc+=1
    accuraccy_list_jacc.append(jacc/num_topics)
avg_acc_list_jacc = sum(accuraccy_list_jacc)/len(y_pred)
print(f"accuracy (jaccard) = {avg_acc_list_jacc:.2f}")

accuracy (bleu) = 0.56
accuracy (rouge) = 0.59
accuracy (jaccard) = 0.79


In [79]:
metrics.multilabel_confusion_matrix(y_true, y_pred)
label_names = list(topic_dict_8.values())
print(metrics.classification_report(y_true, y_pred,target_names=label_names))

                      precision    recall  f1-score   support

            contract       0.31      0.82      0.45       919
personal_right/court       0.76      0.32      0.45      2052
   family/succession       0.89      0.91      0.90      1963
            contract       0.42      0.76      0.54      1042
            criminal       0.26      0.71      0.38       533
            contract       0.75      0.12      0.20      1650
               labor       0.78      0.49      0.60      1017
               labor       0.28      0.45      0.35       579

           micro avg       0.51      0.54      0.53      9755
           macro avg       0.56      0.57      0.48      9755
        weighted avg       0.65      0.54      0.51      9755
         samples avg       0.56      0.59      0.54      9755



In [80]:
for i in range(len(y_pred)):
    if(y_pred[i][0]==0): y_pred[i][0]=y_pred[i][3]
    if(y_pred[i][0]==0): y_pred[i][0]=y_pred[i][5]
    if(y_pred[i][6]==0): y_pred[i][6]=y_pred[i][7]
y_pred = np.delete(y_pred, 7, axis=1)
y_pred = np.delete(y_pred, 5, axis=1)
y_pred = np.delete(y_pred, 3, axis=1)

for i in range(len(y_true)):
    if(y_true[i][1]==0): y_true[i][0]=y_true[i][3]
    if(y_true[i][0]==0): y_true[i][0]=y_true[i][5]
    if(y_true[i][6]==0): y_true[i][6]=y_true[i][7]
y_true = np.delete(y_true, 7, axis=1)
y_true = np.delete(y_true, 5, axis=1)
y_true = np.delete(y_true, 3, axis=1)

metrics.multilabel_confusion_matrix(y_true, y_pred)
label_names = list(topic_dict_8.values())
label_names.pop(7)
label_names.pop(5)
label_names.pop(3)


topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]
topic_true_4 = [sublist[4] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]
topic_pred_4 = [sublist[4] for sublist in y_pred]

print(f'accuracy for topic {label_names[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {label_names[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {label_names[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {label_names[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')
print(f'accuracy for topic {label_names[4]} : {metrics.accuracy_score(topic_true_4, topic_pred_4):.2f}')


print(metrics.classification_report(y_true, y_pred,target_names=label_names))

accuracy for topic contract : 0.72
accuracy for topic personal_right/court : 0.66
accuracy for topic family/succession : 0.93
accuracy for topic criminal : 0.78
accuracy for topic labor : 0.82
                      precision    recall  f1-score   support

            contract       0.53      0.90      0.67      2153
personal_right/court       0.76      0.32      0.45      2052
   family/succession       0.89      0.91      0.90      1963
            criminal       0.26      0.71      0.38       533
               labor       0.68      0.61      0.65      1475

           micro avg       0.61      0.69      0.65      8176
           macro avg       0.62      0.69      0.61      8176
        weighted avg       0.69      0.69      0.64      8176
         samples avg       0.66      0.73      0.66      8176



c:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


num_topics = 9

In [49]:
with open("lda/lda_model_9.pkl", "rb") as f:
    lda_model_9 = pickle.load(f)
with open(f"lda/topic_dict_9.pkl", "rb") as f:
    topic_dict_9 = pickle.load(f)
num_topics = len(topic_dict_9)
pred_question_score = [lda_model_9[text] for text in corpus_question_train]
pred_answer_score = [lda_model_9[text] for text in corpus_answer_train]
threshold = 1/num_topics
question_predict=[]
for each_topic in pred_question_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold): temp_pred.append(1)
    else: temp_pred.append(0)
  question_predict.append(temp_pred)
pd.DataFrame(question_predict)
answer_predict=[]
for each_topic in pred_answer_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold) : temp_pred.append(1)
    else: temp_pred.append(0)
  answer_predict.append(temp_pred)
pd.DataFrame(answer_predict)
y_true = np.array(answer_predict)
y_pred = np.array(question_predict)
topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]
topic_true_4 = [sublist[4] for sublist in y_true]
topic_true_5 = [sublist[5] for sublist in y_true]
topic_true_6 = [sublist[6] for sublist in y_true]
topic_true_7 = [sublist[7] for sublist in y_true]
topic_true_8 = [sublist[8] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]
topic_pred_4 = [sublist[4] for sublist in y_pred]
topic_pred_5 = [sublist[5] for sublist in y_pred]
topic_pred_6 = [sublist[6] for sublist in y_pred]
topic_pred_7 = [sublist[7] for sublist in y_pred]
topic_pred_8 = [sublist[8] for sublist in y_pred]

print(f'accuracy for topic {topic_dict_9[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {topic_dict_9[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {topic_dict_9[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {topic_dict_9[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')
print(f'accuracy for topic {topic_dict_9[4]} : {metrics.accuracy_score(topic_true_4, topic_pred_4):.2f}')
print(f'accuracy for topic {topic_dict_9[5]} : {metrics.accuracy_score(topic_true_5, topic_pred_5):.2f}')
print(f'accuracy for topic {topic_dict_9[6]} : {metrics.accuracy_score(topic_true_6, topic_pred_6):.2f}')
print(f'accuracy for topic {topic_dict_9[7]} : {metrics.accuracy_score(topic_true_7, topic_pred_7):.2f}')
print(f'accuracy for topic {topic_dict_9[8]} : {metrics.accuracy_score(topic_true_8, topic_pred_8):.2f}')

accuracy for topic contract : 0.75
accuracy for topic personal_right/court : 0.69
accuracy for topic family/succession : 0.90
accuracy for topic contract : 0.78
accuracy for topic criminal : 0.78
accuracy for topic contract : 0.73
accuracy for topic family/succession : 0.82
accuracy for topic personal_right/court : 0.84
accuracy for topic labor : 0.93


In [50]:
y_true = answer_predict
y_pred = question_predict

count_true = 0
count_false = 0
pred_list = []
true_list = []
checker = False
for i in range(len(y_true)):
    checker = False
    for j in range(num_topics):
        if(y_true[i][j] == 1):
            if(y_true[i][j] == y_pred[i][j]):
                checker = True
    if(checker == True):
        count_true += 1
        pred_list.append(y_pred[i])
        true_list.append(y_true[i])
    if(checker == False):
        count_false += 1
print(f'count_true = {count_true} docs\ncount_false = {count_false} docs')
print(f"accuracy = {count_true/(count_true+count_false):.2f}")

count_true = 4649 docs
count_false = 976 docs
accuracy = 0.83


In [22]:
y_true = answer_predict
y_pred = question_predict
accuracy_list_bleu = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_pred[i][j]==1):
            count += 1
            if(y_true[i][j]==1):
                check += 1
    accuracy_list_bleu.append(check/count)
avg_acc_list_bleu = sum(accuracy_list_bleu)/len(y_pred)
print(f"accuracy (bleu) = {avg_acc_list_bleu:.2f}")
accuracy_list_rouge = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_true[i][j]==1):
            count += 1
            if(y_pred[i][j]==1):
                check += 1
    accuracy_list_rouge.append(check/count)
avg_acc_list_rouge = sum(accuracy_list_rouge)/len(y_pred)
print(f"accuracy (rouge) = {avg_acc_list_rouge:.2f}")

accuraccy_list_jacc = []
for i in range(len(y_pred)):
    jacc = 0
    for j in range(num_topics):
        if(y_true[i][j]==y_pred[i][j]): jacc+=1
    accuraccy_list_jacc.append(jacc/num_topics)
avg_acc_list_jacc = sum(accuraccy_list_jacc)/len(y_pred)
print(f"accuracy (jaccard) = {avg_acc_list_jacc:.2f}")

accuracy (bleu) = 0.57
accuracy (rouge) = 0.58
accuracy (jaccard) = 0.80


In [84]:
metrics.multilabel_confusion_matrix(y_true, y_pred)
label_names = list(topic_dict_9.values())
print(metrics.classification_report(y_true, y_pred,target_names=label_names))

                      precision    recall  f1-score   support

            contract       0.33      0.81      0.47       932
personal_right/court       0.71      0.30      0.42      1744
   family/succession       0.78      0.91      0.84      1670
            contract       0.42      0.72      0.53       982
            criminal       0.26      0.70      0.38       545
            contract       0.79      0.16      0.26      1732
   family/succession       0.55      0.22      0.31      1051
personal_right/court       0.31      0.33      0.32       643
               labor       0.78      0.84      0.81       964

           micro avg       0.51      0.53      0.52     10263
           macro avg       0.55      0.55      0.48     10263
        weighted avg       0.61      0.53      0.49     10263
         samples avg       0.57      0.58      0.54     10263



In [85]:
for i in range(len(y_pred)):
    if(y_pred[i][0]==0): y_pred[i][0]=y_pred[i][3]
    if(y_pred[i][0]==0): y_pred[i][0]=y_pred[i][5]
    if(y_pred[i][1]==0): y_pred[i][1]=y_pred[i][7]
    if(y_pred[i][2]==0): y_pred[i][2]=y_pred[i][6]
y_pred = np.delete(y_pred, 7, axis=1)
y_pred = np.delete(y_pred, 6, axis=1)
y_pred = np.delete(y_pred, 5, axis=1)
y_pred = np.delete(y_pred, 3, axis=1)

for i in range(len(y_true)):
    if(y_true[i][0]==0): y_true[i][0]=y_true[i][3]
    if(y_true[i][0]==0): y_true[i][0]=y_true[i][5]
    if(y_true[i][1]==0): y_true[i][1]=y_true[i][6]
    if(y_true[i][2]==0): y_true[i][2]=y_true[i][7]
y_true = np.delete(y_true, 7, axis=1)
y_true = np.delete(y_true, 6, axis=1)
y_true = np.delete(y_true, 5, axis=1)
y_true = np.delete(y_true, 3, axis=1)

metrics.multilabel_confusion_matrix(y_true, y_pred)
label_names = list(topic_dict_9.values())
label_names.pop(7)
label_names.pop(6)
label_names.pop(5)
label_names.pop(3)


topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]
topic_true_4 = [sublist[4] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]
topic_pred_4 = [sublist[4] for sublist in y_pred]

print(f'accuracy for topic {label_names[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {label_names[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {label_names[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {label_names[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')
print(f'accuracy for topic {label_names[4]} : {metrics.accuracy_score(topic_true_4, topic_pred_4):.2f}')


print(metrics.classification_report(y_true, y_pred,target_names=label_names))

accuracy for topic contract : 0.61
accuracy for topic personal_right/court : 0.74
accuracy for topic family/succession : 0.79
accuracy for topic criminal : 0.78
accuracy for topic labor : 0.93
                      precision    recall  f1-score   support

            contract       0.67      0.88      0.76      2632
personal_right/court       0.67      0.34      0.45      2645
   family/succession       0.73      0.72      0.73      2214
            criminal       0.26      0.70      0.38       545
               labor       0.78      0.84      0.81       964

           micro avg       0.63      0.67      0.65      9000
           macro avg       0.62      0.70      0.63      9000
        weighted avg       0.67      0.67      0.64      9000
         samples avg       0.70      0.72      0.67      9000



num_topics = 10

In [51]:
with open("lda/lda_model_10.pkl", "rb") as f:
    lda_model_10 = pickle.load(f)
with open(f"lda/topic_dict_10.pkl", "rb") as f:
    topic_dict_10 = pickle.load(f)
num_topics = len(topic_dict_10)
pred_question_score = [lda_model_10[text] for text in corpus_question_train]
pred_answer_score = [lda_model_10[text] for text in corpus_answer_train]
threshold = 1/num_topics
question_predict=[]
for each_topic in pred_question_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold): temp_pred.append(1)
    else: temp_pred.append(0)
  question_predict.append(temp_pred)
pd.DataFrame(question_predict)
answer_predict=[]
for each_topic in pred_answer_score:
  temp_pred = []
  for topic in each_topic:
    if(topic[1]>threshold) : temp_pred.append(1)
    else: temp_pred.append(0)
  answer_predict.append(temp_pred)
pd.DataFrame(answer_predict)
y_true = np.array(answer_predict)
y_pred = np.array(question_predict)
topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]
topic_true_4 = [sublist[4] for sublist in y_true]
topic_true_5 = [sublist[5] for sublist in y_true]
topic_true_6 = [sublist[6] for sublist in y_true]
topic_true_7 = [sublist[7] for sublist in y_true]
topic_true_8 = [sublist[8] for sublist in y_true]
topic_true_9 = [sublist[9] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]
topic_pred_4 = [sublist[4] for sublist in y_pred]
topic_pred_5 = [sublist[5] for sublist in y_pred]
topic_pred_6 = [sublist[6] for sublist in y_pred]
topic_pred_7 = [sublist[7] for sublist in y_pred]
topic_pred_8 = [sublist[8] for sublist in y_pred]
topic_pred_9 = [sublist[9] for sublist in y_pred]

print(f'accuracy for topic {topic_dict_10[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {topic_dict_10[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {topic_dict_10[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {topic_dict_10[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')
print(f'accuracy for topic {topic_dict_10[4]} : {metrics.accuracy_score(topic_true_4, topic_pred_4):.2f}')
print(f'accuracy for topic {topic_dict_10[5]} : {metrics.accuracy_score(topic_true_5, topic_pred_5):.2f}')
print(f'accuracy for topic {topic_dict_10[6]} : {metrics.accuracy_score(topic_true_6, topic_pred_6):.2f}')
print(f'accuracy for topic {topic_dict_10[7]} : {metrics.accuracy_score(topic_true_7, topic_pred_7):.2f}')
print(f'accuracy for topic {topic_dict_10[8]} : {metrics.accuracy_score(topic_true_8, topic_pred_8):.2f}')
print(f'accuracy for topic {topic_dict_10[9]} : {metrics.accuracy_score(topic_true_8, topic_pred_9):.2f}')

accuracy for topic contract : 0.75
accuracy for topic personal_right/court : 0.70
accuracy for topic family/succession : 0.90
accuracy for topic contract : 0.66
accuracy for topic criminal : 0.80
accuracy for topic contract : 0.90
accuracy for topic family/succession : 0.85
accuracy for topic personal_right/court : 0.84
accuracy for topic labor : 0.93
accuracy for topic contract : 0.76


In [52]:
y_true = answer_predict
y_pred = question_predict

count_true = 0
count_false = 0
pred_list = []
true_list = []
checker = False
for i in range(len(y_true)):
    checker = False
    for j in range(num_topics):
        if(y_true[i][j] == 1):
            if(y_true[i][j] == y_pred[i][j]):
                checker = True
    if(checker == True):
        count_true += 1
        pred_list.append(y_pred[i])
        true_list.append(y_true[i])
    if(checker == False):
        count_false += 1
print(f'count_true = {count_true} docs\ncount_false = {count_false} docs')
print(f"accuracy = {count_true/(count_true+count_false):.2f}")

count_true = 4494 docs
count_false = 1131 docs
accuracy = 0.80


In [34]:
y_true = answer_predict
y_pred = question_predict
accuracy_list_bleu = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_pred[i][j]==1):
            count += 1
            if(y_true[i][j]==1):
                check += 1
    accuracy_list_bleu.append(check/count)
avg_acc_list_2 = sum(accuracy_list_bleu)/len(y_pred)
print(f"accuracy (bleu) = {avg_acc_list_2:.2f}")
accuracy_list_rouge = []
for i in range(len(y_pred)):
    count = 0
    check = 0
    for j in range(num_topics):
        if(y_true[i][j]==1):
            count += 1
            if(y_pred[i][j]==1):
                check += 1
    accuracy_list_rouge.append(check/count)
avg_acc_list_2 = sum(accuracy_list_rouge)/len(y_pred)
print(f"accuracy (rouge) = {avg_acc_list_2:.2f}")

accuraccy_list_jacc = []
for i in range(len(y_pred)):
    jacc = 0
    for j in range(num_topics):
        if(y_true[i][j]==y_pred[i][j]): jacc+=1
    accuraccy_list_jacc.append(jacc/num_topics)
avg_acc_list_jacc = sum(accuraccy_list_jacc)/len(y_pred)
print(f"accuracy (jaccard) = {avg_acc_list_jacc:.2f}")

accuracy (bleu) = 0.53
accuracy (rouge) = 0.55
accuracy (jaccard) = 0.80


In [95]:
metrics.multilabel_confusion_matrix(y_true, y_pred)
label_names = list(topic_dict_10.values())
print(metrics.classification_report(y_true, y_pred,target_names=label_names))

                      precision    recall  f1-score   support

            contract       0.33      0.79      0.46       936
personal_right/court       0.66      0.29      0.40      1652
   family/succession       0.82      0.86      0.84      1714
            contract       0.19      0.77      0.30       533
            criminal       0.29      0.66      0.40       570
            contract       0.46      0.23      0.30       565
   family/succession       0.53      0.28      0.36       888
personal_right/court       0.33      0.30      0.32       702
               labor       0.78      0.82      0.80       988
            contract       0.83      0.18      0.29      2059

           micro avg       0.47      0.49      0.48     10607
           macro avg       0.52      0.52      0.45     10607
        weighted avg       0.62      0.49      0.47     10607
         samples avg       0.53      0.55      0.50     10607



In [96]:
for i in range(len(y_pred)):
    if(y_pred[i][0]==0): y_pred[i][0]=y_pred[i][3]
    if(y_pred[i][0]==0): y_pred[i][0]=y_pred[i][5]
    if(y_pred[i][0]==0): y_pred[i][0]=y_pred[i][9]
    if(y_pred[i][1]==0): y_pred[i][1]=y_pred[i][7]
    if(y_pred[i][2]==0): y_pred[i][2]=y_pred[i][6]
y_pred = np.delete(y_pred, 9, axis=1)
y_pred = np.delete(y_pred, 7, axis=1)
y_pred = np.delete(y_pred, 6, axis=1)
y_pred = np.delete(y_pred, 5, axis=1)
y_pred = np.delete(y_pred, 3, axis=1)

for i in range(len(y_true)):
    if(y_true[i][0]==0): y_true[i][0]=y_true[i][3]
    if(y_true[i][0]==0): y_true[i][0]=y_true[i][5]
    if(y_true[i][0]==0): y_true[i][0]=y_true[i][9]
    if(y_true[i][1]==0): y_true[i][1]=y_true[i][6]
    if(y_true[i][2]==0): y_true[i][2]=y_true[i][7]
y_true = np.delete(y_true, 9, axis=1)
y_true = np.delete(y_true, 7, axis=1)
y_true = np.delete(y_true, 6, axis=1)
y_true = np.delete(y_true, 5, axis=1)
y_true = np.delete(y_true, 3, axis=1)

metrics.multilabel_confusion_matrix(y_true, y_pred)
label_names = list(topic_dict_10.values())
label_names.pop(9)
label_names.pop(7)
label_names.pop(6)
label_names.pop(5)
label_names.pop(3)


topic_true_0 = [sublist[1] for sublist in y_true]
topic_true_1 = [sublist[0] for sublist in y_true]
topic_true_2 = [sublist[2] for sublist in y_true]
topic_true_3 = [sublist[3] for sublist in y_true]
topic_true_4 = [sublist[4] for sublist in y_true]

topic_pred_0 = [sublist[1] for sublist in y_pred]
topic_pred_1 = [sublist[0] for sublist in y_pred]
topic_pred_2 = [sublist[2] for sublist in y_pred]
topic_pred_3 = [sublist[3] for sublist in y_pred]
topic_pred_4 = [sublist[4] for sublist in y_pred]

print(f'accuracy for topic {label_names[0]} : {metrics.accuracy_score(topic_true_0, topic_pred_0):.2f}')
print(f'accuracy for topic {label_names[1]} : {metrics.accuracy_score(topic_true_1, topic_pred_1):.2f}')
print(f'accuracy for topic {label_names[2]} : {metrics.accuracy_score(topic_true_2, topic_pred_2):.2f}')
print(f'accuracy for topic {label_names[3]} : {metrics.accuracy_score(topic_true_3, topic_pred_3):.2f}')
print(f'accuracy for topic {label_names[4]} : {metrics.accuracy_score(topic_true_4, topic_pred_4):.2f}')


print(metrics.classification_report(y_true, y_pred,target_names=label_names))

accuracy for topic contract : 0.63
accuracy for topic personal_right/court : 0.71
accuracy for topic family/succession : 0.78
accuracy for topic criminal : 0.80
accuracy for topic labor : 0.93
                      precision    recall  f1-score   support

            contract       0.66      0.88      0.76      2912
personal_right/court       0.63      0.33      0.43      2400
   family/succession       0.76      0.69      0.72      2312
            criminal       0.29      0.66      0.40       570
               labor       0.78      0.82      0.80       988

           micro avg       0.64      0.67      0.65      9182
           macro avg       0.62      0.68      0.62      9182
        weighted avg       0.67      0.67      0.65      9182
         samples avg       0.70      0.72      0.67      9182

